In [15]:
import pandas as pd
import requests
import gzip
import json
import io

Read json file in

In [16]:
url = 'https://mcauleylab.ucsd.edu/public_datasets/gdrive/googlelocal/review-Vermont_10.json.gz'

response = requests.get(url, stream = True)
response.raise_for_status() 

with gzip.GzipFile(fileobj = io.BytesIO(response.content), mode = 'rb') as gz_file:
    data_list = [json.loads(line) for line in gz_file]

vt = pd.DataFrame(data_list)

In [17]:
print(vt.shape)
print(vt.columns)
# print(vt.head())

(324725, 8)
Index(['user_id', 'name', 'time', 'rating', 'text', 'pics', 'resp', 'gmap_id'], dtype='object')


In [18]:
url_metadata = 'https://mcauleylab.ucsd.edu/public_datasets/gdrive/googlelocal/meta-Vermont.json.gz'

response_metadata = requests.get(url_metadata, stream = True)
response_metadata.raise_for_status() 

with gzip.GzipFile(fileobj = io.BytesIO(response_metadata.content), mode = 'rb') as gz_file:
    data_list1 = [json.loads(line) for line in gz_file]

vt_metadata = pd.DataFrame(data_list1)

In [19]:
print(vt_metadata.shape)
print(vt_metadata.columns)
# print(vt_metadata.head())

(11291, 15)
Index(['name', 'address', 'gmap_id', 'description', 'latitude', 'longitude',
       'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'MISC',
       'state', 'relative_results', 'url'],
      dtype='object')


Data Cleaning

For business data
Drop 'address', 'latitude', 'longitude', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'MISC', 'state', 'relative_results' column, convert all text to lower case, drop duplicates (around 50 duplicates)

In [20]:
print(vt_metadata.shape)
print(vt_metadata.columns)
vt_metadata = vt_metadata.drop(['address', 'latitude', 'longitude', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'MISC', 'state', 'relative_results'], axis = 1)
vt_metadata.columns = vt_metadata.columns.str.lower() 
vt_metadata['description'] = vt_metadata['description'].str.replace(r'\s+', ' ', regex=True)
vt_metadata['category'] = vt_metadata['category'].str.replace(r'\s+', ' ', regex=True)
vt_metadata = vt_metadata.drop_duplicates(subset = ['name','gmap_id', 'gmap_id'])

(11291, 15)
Index(['name', 'address', 'gmap_id', 'description', 'latitude', 'longitude',
       'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'MISC',
       'state', 'relative_results', 'url'],
      dtype='object')


For Review data drop 'name' column, convert all text to lower case, drop duplicates (~3000 duplicates)

In [9]:
vt = vt.drop(['name', 'resp'], axis = 1)
vt.columns = vt.columns.str.lower() 
vt['text'] = vt['text'].str.replace(r'\s+', ' ', regex=True)
vt = vt.drop_duplicates(subset = ['user_id', 'text', 'gmap_id', 'time'])

Change the format of the 'pics' column, merge multiple dictionaries into one with list of pic urls

In [21]:
# vt_pics = vt[vt['pics'].notna()]
# print(vt_pics.head())
# print(vt_pics['pics'])
# import pprint
# pprint.pprint(vt_pics.loc[17, 'pics'])

In [22]:
def collapse_pics(pic_list):
    if not pic_list:
        return []  
    urls = []
    for pic_dict in pic_list:
        urls.extend(pic_dict.get('url', []))
    return urls

vt['pics_collapsed'] = vt['pics'].apply(collapse_pics)

# print(vt['pics_collapsed'].iloc[17])

Merge by gmap id

In [23]:
vt_merged = pd.merge(vt, vt_metadata, on = 'gmap_id', how = 'inner')

Separate dataset into vt1 (rating only) and vt2 (everything else)

In [24]:
vt1 = vt_merged[vt_merged['pics'].isna() & vt_merged['text'].isna()] # rating only
vt2 = vt_merged[~vt_merged.index.isin(vt1.index)] # everything else
vt1 = vt1.drop(['pics'], axis = 1)
vt2 = vt2.drop(['pics'], axis = 1)
print(vt1.shape)
print(vt2.shape)

(145637, 12)
(179088, 12)


In [25]:
# vt1.to_csv('vermont_merged_rating only.csv')
vt2.to_csv('vermont_merged_text_and_or_review.csv')
# vt.to_csv('vermont.csv')
# vt_merged.to_csv('vermont_merged.csv')
# vt_metadata.to_csv('vermont_metadata.csv')